In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
import joblib

In [2]:
# CSV 파일 로드
df_ingredients = pd.read_csv('../data/raw/ingredients.csv')
df_ingredient_features = pd.read_csv("../data/final/ingredient_features.csv", index_col=0, encoding="utf-8")
df_ingredient_features_eng = pd.read_csv("../data/final/ingredient_features_eng.csv", index_col=0, encoding="utf-8")
df_drinks = pd.read_csv("../data/final/drinks_flavor_balanced.csv", encoding="cp949") # 칵테일 이름과 점수 정보를 담은 파일

In [17]:
df_drinks

,id,name,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
0,1,A Day at the Beach,15,7,3,2,4,0,0,7,3,0,0,0,8,0,0,0,0
1,2,A True Amaretto Sour,20,8,7,2,4,0,0,5,2,0,0,0,9,0,0,8,0
2,3,A1,25,6,3,2,5,0,0,6,2,0,0,0,0,0,0,0,8
3,4,Abbey Cocktail,25,5,3,4,4,0,0,4,2,0,0,0,0,0,0,0,8
4,5,Abbey Martini,35,3,1,4,5,3,0,1,1,0,0,0,0,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,258,Wine Cooler,10,7,3,1,2,0,0,5,5,0,0,0,8,0,0,0,0
258,259,Wine Punch,12,8,2,1,3,0,0,7,6,0,0,0,9,0,0,0,0
259,260,Winter Rita,14,6,6,1,3,0,0,5,7,0,0,0,0,0,0,0,0
260,261,Yellow Bird,25,4,2,2,4,1,0,5,4,0,0,0,0,0,0,0,0


In [19]:
df_ingredients

,id,ingredient_name
0,1,1 oz rum
1,1,0.5 oz Almond Liqueur
2,1,4 oz Orange juice
3,1,Simple Syrup
4,2,1 oz Almond Liqueur
...,...,...
966,261,1.5 cl Lime Juice
967,262,5 cl Orange Liqueur
968,262,2 cl Lemon juice
969,262,ice


In [21]:
df_ingredient_features

,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
ingredients,,,,,,,,,,,,,,,,,
진,40,0,0,2,4,9,0,0,1,0,0,0,0,0,0,0,4
럼,40,1,0,1,4,0,0,2,0,0,0,4,4,0,0,0,4
보드카,40,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,10
위스키,40,1,0,3,6,2,1,0,0,0,0,0,0,8,4,5,0
브랜디,40,2,0,1,5,1,0,2,0,0,0,0,0,6,3,6,0
데킬라,40,0,1,2,4,2,0,0,1,0,0,0,0,3,8,0,0
압생트,40,1,0,4,5,9,0,0,0,0,0,0,0,0,0,0,0
문샤인,45,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,7
오렌지 리큐르,30,8,1,2,3,1,0,8,6,0,0,0,7,0,0,0,0


In [11]:
df_garnish_rows = pd.DataFrame([
    {"ingredients": "레몬 필", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "레몬 웨지", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "레몬 슬라이스", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "라임 필", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "라임 웨지", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "라임 슬라이스", "abv": 0, "sweetness": 0, "sourness": 2, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 10, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "오렌지 필", "abv": 0, "sweetness": 1, "sourness": 1, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 9, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "오렌지 웨지", "abv": 0, "sweetness": 1, "sourness": 1, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 9, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "오렌지 슬라이스", "abv": 0, "sweetness": 1, "sourness": 1, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 9, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
    {"ingredients": "체리", "abv": 0, "sweetness": 4, "sourness": 0, "bitterness": 0, "body": 0, "herbal": 0, "coffee": 0, "fruity": 0, "citrus": 0, "creamy": 0, "berry": 10, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 5},
    {"ingredients": "애플민트", "abv": 0, "sweetness": 0, "sourness": 0, "bitterness": 1, "body": 0, "herbal": 10, "coffee": 0, "fruity": 0, "citrus": 0, "creamy": 0, "berry": 0, "tropical": 0, "sweet": 0, "smoky": 0, "spicy": 0, "nutty": 0, "plain": 8},
])

df_garnish_rows.set_index("ingredients", inplace=True)

In [7]:
df_garnish_rows

,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
ingredients,,,,,,,,,,,,,,,,,
레몬 필,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
레몬 웨지,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
레몬 슬라이스,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
라임 필,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
라임 웨지,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
라임 슬라이스,0,0,2,0,0,0,0,0,10,0,0,0,0,0,0,0,8
오렌지 필,0,1,1,0,0,0,0,0,9,0,0,0,0,0,0,0,8
오렌지 웨지,0,1,1,0,0,0,0,0,9,0,0,0,0,0,0,0,8
오렌지 슬라이스,0,1,1,0,0,0,0,0,9,0,0,0,0,0,0,0,8


In [8]:
# 병합
df_ingredient_features_2 = pd.concat([df_ingredient_features, df_garnish_rows])

In [9]:
df_ingredient_features_2

,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
ingredients,,,,,,,,,,,,,,,,,
진,40,0,0,2,4,9,0,0,1,0,0,0,0,0,0,0,4
럼,40,1,0,1,4,0,0,2,0,0,0,4,4,0,0,0,4
보드카,40,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,10
위스키,40,1,0,3,6,2,1,0,0,0,0,0,0,8,4,5,0
브랜디,40,2,0,1,5,1,0,2,0,0,0,0,0,6,3,6,0
데킬라,40,0,1,2,4,2,0,0,1,0,0,0,0,3,8,0,0
압생트,40,1,0,4,5,9,0,0,0,0,0,0,0,0,0,0,0
문샤인,45,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,7
오렌지 리큐르,30,8,1,2,3,1,0,8,6,0,0,0,7,0,0,0,0


In [25]:
# 저장
df_ingredient_features.to_csv("ingredient_features_with_garnish.csv", index=False)

In [27]:
ingredient_mapping = {
    '진': 'gin',
    '럼': 'rum',
    '보드카': 'vodka',
    '위스키': 'whiskey',
    '브랜디': 'brandy',
    '데킬라': 'tequila',
    '압생트': 'absinthe',
    '문샤인': 'moonshine',
    '오렌지 리큐르': 'orange_liqueur',
    '블루 큐라소': 'blue_curacao',
    '오렌지 주스': 'orange_juice',
    '라임 주스': 'lime_juice',
    '레몬 주스': 'lemon_juice',
    '체리 리큐르': 'cherry_liqueur',
    '크랜베리 주스': 'cranberry_juice',
    '파인애플 주스': 'pineapple_juice',
    '아몬드 리큐르': 'almond_liqueur',
    '심플 시럽': 'simple_syrup',
    '커피 리큐르': 'coffee_liqueur',
    '허브 리큐르': 'herbal_liqueur',
    '드라이 베르무트': 'dry_vermouth',
    '스위트 베르무트': 'sweet_vermouth',
    '비터스': 'bitters',
    '우유': 'milk',
    '탄산수': 'soda',
    '그레나딘 시럽': 'grenadine_syrup',

    # 🍋 Garnish 추가
    '레몬 필': 'lemon_peel',
    '레몬 웨지': 'lemon_wedge',
    '레몬 슬라이스': 'lemon_slice',
    '라임 필': 'lime_peel',
    '라임 웨지': 'lime_wedge',
    '라임 슬라이스': 'lime_slice',
    '오렌지 필': 'orange_peel',
    '오렌지 웨지': 'orange_wedge',
    '오렌지 슬라이스': 'orange_slice',
    '체리': 'cherry',
    '애플민트': 'apple_mint'
}


In [29]:
def convert_to_ml(amount_str):
    if not amount_str or pd.isna(amount_str) or amount_str == "":
        return 5  # 양이 명시되지 않은 경우 기본값
    
    # "Fill", "Top", "Filled" 등의 표현이 있으면 큰 양으로 처리
    if re.search(r'fill|top|full', str(amount_str).lower()):
        return 60  # filling은 60ml로 가정
    
    # 숫자 추출 (분수 포함)
    amount_match = re.search(r'(\d+(?:\.\d+)?|\d+/\d+)', str(amount_str))
    if not amount_match:
        return 5  # 숫자가 없는 경우 기본값
    
    amount_val = amount_match.group(1)
    # 분수 처리 (1/2 -> 0.5)
    if '/' in amount_val:
        num, denom = amount_val.split('/')
        amount_num = float(num) / float(denom)
    else:
        amount_num = float(amount_val)
    
    # 단위에 따른 변환
    amount_str_lower = str(amount_str).lower()
    if 'oz' in amount_str_lower:
        return amount_num * 30  # 1 oz = 30 ml
    elif 'dash' in amount_str_lower:
        return amount_num * 5   # 1 dash = 5 ml
    elif 'shot' in amount_str_lower:
        return amount_num * 45  # 1 shot = 45 ml
    elif 'jigger' in amount_str_lower:
        return amount_num * 45  # 1 jigger = 45 ml
    elif 'tsp' in amount_str_lower:
        return amount_num * 5   # 1 tsp = 5 ml
    elif 'tblsp' in amount_str_lower or 'tbsp' in amount_str_lower:
        return amount_num * 15  # 1 tablespoon = 15 ml
    elif 'part' in amount_str_lower:
        return amount_num * 30  # 1 part = 30 ml (상대적 비율)
    elif 'cl' in amount_str_lower:
        return amount_num * 10  # 1 cl = 10 ml
    elif 'ml' in amount_str_lower:
        return amount_num  # 이미 ml 단위일 경우 그대로 사용
    else:
        return amount_num * 30  # 기본값으로 30ml 가정

In [31]:
# 재료량과 재료이름 분리하기 위한 개선된 함수
def extract_amount_and_ingredient(ingredient_text):
    if pd.isna(ingredient_text):
        return "", ""
    
    # "Juice of 1/2 Lemon"과 같은 특수 케이스 처리
    juice_match = re.match(r'juice\s+of\s+(.+?)\s+(.+)', ingredient_text.lower())
    if juice_match:
        amount = juice_match.group(1)
        ingredient = juice_match.group(2)
        # "juice of 1/2 lemon" -> 양: "1/2", 재료: "lemon juice"
        if ingredient.lower() in ['lemon', 'lime', 'orange']:
            return amount, f"{ingredient} juice"
        return amount, ingredient
    
    # 일반적인 패턴: "1 oz Rum", "1/2 shot Gin" 등
    pattern = r'^((?:[\d./]+)?\s*(?:oz|shot|tsp|tblsp|tbsp|dash|jigger|part|fill|cl|ml|top|can)?\s*(?:of)?)\s*(.+)$'
    match = re.match(pattern, ingredient_text.strip(), re.IGNORECASE)
    
    if match:
        amount = match.group(1).strip()
        ingredient = match.group(2).strip()
        # 양이 비어있으면 빈 문자열 반환
        if amount == "":
            return "", ingredient
        return amount, ingredient
    else:
        # 매칭되지 않으면 전체를 재료명으로 간주
        return "", ingredient_text.strip()

In [33]:
ingredient_mapping.update({
    'mint_leaves': 'apple_mint',
    'mint_garnish': 'apple_mint',
    'orange_garnish': 'orange_peel',  # 또는 'orange_slice'
    'lime_garnish': 'lime_peel',      # 또는 'lime_slice'
    'cherry_garnish': 'cherry',
    'water': 'soda',
    'ice': 'ice',
    'salt': 'salt',
    'olive_garnish': 'olive_garnish'
})

# 매핑 불가한 재료만
special_ingredients = ['ice', 'salt', 'olive_garnish']

# 기본 양
special_amounts = {
    'ice': 100,
    'salt': 2,
    'olive_garnish': 5
}

In [35]:
# 재료를 표준화하는 함수 (매핑 사전 사용)
def standardize_ingredient(ingredient):
    if pd.isna(ingredient):
        return ""
    
    # 소문자로 변환
    ingredient_lower = ingredient.lower()
    
    # 매핑 사전에서 찾기
    for key, value in ingredient_mapping.items():
        if key in ingredient_lower:
            return value
    
    # 매핑이 없으면 공백을 언더스코어로 변경
    return ingredient_lower.replace(' ', '_')

# 새로운 칼럼 추가
df_ingredients[['amount_str', 'ingredient']] = df_ingredients['ingredient_name'].apply(
    lambda x: pd.Series(extract_amount_and_ingredient(x))
)

# 재료 표준화 적용
df_ingredients['std_ingredient'] = df_ingredients['ingredient'].apply(standardize_ingredient)

# 재료량을 ml로 변환
df_ingredients['amount_ml'] = df_ingredients['amount_str'].apply(convert_to_ml)

# 특별 재료의 경우 기본 양 적용
for i, row in df_ingredients.iterrows():
    if row['std_ingredient'] in special_ingredients:
        # 이미 양이 지정되어 있으면 그대로 사용, 아니면 기본값 사용
        if pd.isna(row['amount_str']) or row['amount_str'] == "":
            df_ingredients.at[i, 'amount_ml'] = special_amounts[row['std_ingredient']]

In [37]:
# 처리된 데이터 확인
print("처리된 데이터 샘플:")
print(df_ingredients[['ingredient_name', 'amount_str', 'ingredient', 'std_ingredient', 'amount_ml']].head(10))

처리된 데이터 샘플:
          ingredient_name amount_str         ingredient     std_ingredient  \
0               1 oz  rum       1 oz                rum                rum   
1  0.5 oz  Almond Liqueur     0.5 oz     Almond Liqueur     almond_liqueur   
2      4 oz  Orange juice       4 oz       Orange juice                ice   
3            Simple Syrup                  Simple Syrup       simple_syrup   
4    1 oz  Almond Liqueur       1 oz     Almond Liqueur     almond_liqueur   
5                   lemon                         lemon              lemon   
6                     Ice                           Ice                ice   
7       Maraschino cherry             Maraschino cherry  maraschino_cherry   
8             2 shot  Gin     2 shot                Gin                gin   
9  1 Shot  Orange Liqueur     1 Shot     Orange Liqueur     orange_liqueur   

   amount_ml  
0       30.0  
1       15.0  
2      120.0  
3        5.0  
4       30.0  
5        5.0  
6      100.0  
7        

In [39]:
# ID별로 그룹화하여 레시피 딕셔너리 생성
recipes = {}
for id_val, group in df_ingredients.groupby('id'):
    recipes[id_val] = {}
    for _, row in group.iterrows():
        std_ingredient = row['std_ingredient']
        if std_ingredient and std_ingredient != "":  # 빈 문자열 무시
            amount_ml = row['amount_ml']
            recipes[id_val][std_ingredient] = amount_ml

In [41]:
for recipe_id, ingredients in list(recipes.items())[:3]:  # 처음 3개 레시피만 출력
    print(f"\n레시피 ID: {recipe_id}")
    for ingredient, amount in ingredients.items():
        print(f"  - {amount} ml {ingredient}")


레시피 ID: 1
  - 30.0 ml rum
  - 15.0 ml almond_liqueur
  - 120.0 ml ice
  - 5.0 ml simple_syrup

레시피 ID: 2
  - 30.0 ml almond_liqueur
  - 5.0 ml lemon
  - 100.0 ml ice
  - 5.0 ml maraschino_cherry

레시피 ID: 3
  - 90.0 ml gin
  - 45.0 ml orange_liqueur
  - 11.25 ml ice
  - 5.0 ml simple_syrup


In [43]:
# 레시피를 특성 벡터로 변환하는 함수
def recipe_to_vector(recipe, ingredient_features):
    categories = ingredient_features.columns.tolist()
    cocktail_vector = np.zeros(len(categories))  # 벡터 초기화
    
    total_amount = 0
    missing_ingredients = []

    for ingredient, amount_ml in recipe.items():
        if ingredient in ingredient_features.index:  # 재료가 특성 데이터프레임에 있는지 확인
            cocktail_vector += ingredient_features.loc[ingredient].values * amount_ml
            total_amount += amount_ml
        else:
            missing_ingredients.append(ingredient)
    
    # 누락된 재료 보고
    if missing_ingredients:
        print(f"경고: 다음 재료가 특성 데이터프레임에 없습니다: {', '.join(missing_ingredients)}")
    
    # 총량으로 나누어 정규화
    if total_amount > 0:
        cocktail_vector /= total_amount
    else:
        print("경고: 총량이 0입니다. 벡터 값이 0으로 남을 수 있습니다.")
    
    return cocktail_vector

In [45]:
# 'drink.csv'에서 칵테일 특징 점수 로딩
drink_features = ['abv', 'sweetness', 'sourness', 'bitterness', 'body', 'herbal', 'coffee', 'fruity', 'citrus', 'creamy', 'berry', 'tropical', 'sweet', 'smoky', 'spicy', 'nutty', 'plain']

In [47]:
# 훈련 데이터 생성
X = []
y = []
valid_recipes = []

# df_drinks의 구조 확인
print("\n=== drinks_flavor_balanced.csv 구조 확인 ===")
print(f"df_drinks 열: {df_drinks.columns.tolist()}")


=== drinks_flavor_balanced.csv 구조 확인 ===
df_drinks 열: ['id', 'name', 'abv', 'sweetness', 'sourness', 'bitterness', 'body', 'herbal', 'coffee', 'fruity', 'citrus', 'creamy', 'berry', 'tropical', 'sweet', 'smoky', 'spicy', 'nutty', 'plain']


In [49]:
df_drinks.head(5)

,id,name,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
0,1,A Day at the Beach,15,7,3,2,4,0,0,7,3,0,0,0,8,0,0,0,0
1,2,A True Amaretto Sour,20,8,7,2,4,0,0,5,2,0,0,0,9,0,0,8,0
2,3,A1,25,6,3,2,5,0,0,6,2,0,0,0,0,0,0,0,8
3,4,Abbey Cocktail,25,5,3,4,4,0,0,4,2,0,0,0,0,0,0,0,8
4,5,Abbey Martini,35,3,1,4,5,3,0,1,1,0,0,0,0,0,0,0,8


In [51]:
# 인덱스 설정 전에 열 확인
if 'name' not in df_drinks.columns:
    print("오류: 'drinks.csv'에 'name' 열이 없습니다.")
    # 첫 번째 열을 인덱스로 사용하려면:
    first_column = df_drinks.columns[0]
    print(f"첫 번째 열({first_column})을 인덱스로 사용합니다.")
    df_drinks.set_index(first_column, inplace=True)
else:
    df_drinks.set_index('name', inplace=True)

# 인덱스 설정 후 확인
print(f"인덱스 설정 후 drink_df 샘플:\n{df_drinks.head()}")

# drink_df에서 필요한 특성이 있는지 확인
missing_features = [feature for feature in drink_features if feature not in df_drinks.columns]
if missing_features:
    print(f"오류: drinks.csv에 다음 특성이 없습니다: {', '.join(missing_features)}")
    print(f"사용 가능한 열: {df_drinks.columns.tolist()}")

인덱스 설정 후 drink_df 샘플:
                      id  abv  sweetness  sourness  bitterness  body  herbal  \
name                                                                           
A Day at the Beach     1   15          7         3           2     4       0   
A True Amaretto Sour   2   20          8         7           2     4       0   
A1                     3   25          6         3           2     5       0   
Abbey Cocktail         4   25          5         3           4     4       0   
Abbey Martini          5   35          3         1           4     5       3   

                      coffee  fruity  citrus  creamy  berry  tropical  sweet  \
name                                                                           
A Day at the Beach         0       7       3       0      0         0      8   
A True Amaretto Sour       0       5       2       0      0         0      9   
A1                         0       6       2       0      0         0      0   
Abbey Cocktail   

In [53]:
df_ingredients

,id,ingredient_name,amount_str,ingredient,std_ingredient,amount_ml
0,1,1 oz rum,1 oz,rum,rum,30.0
1,1,0.5 oz Almond Liqueur,0.5 oz,Almond Liqueur,almond_liqueur,15.0
2,1,4 oz Orange juice,4 oz,Orange juice,ice,120.0
3,1,Simple Syrup,,Simple Syrup,simple_syrup,5.0
4,2,1 oz Almond Liqueur,1 oz,Almond Liqueur,almond_liqueur,30.0
...,...,...,...,...,...,...
966,261,1.5 cl Lime Juice,1.5 cl,Lime Juice,ice,15.0
967,262,5 cl Orange Liqueur,5 cl,Orange Liqueur,orange_liqueur,50.0
968,262,2 cl Lemon juice,2 cl,Lemon juice,ice,20.0
969,262,ice,,ice,ice,100.0


In [55]:
df_ingredient_features

,abv,sweetness,sourness,bitterness,body,herbal,coffee,fruity,citrus,creamy,berry,tropical,sweet,smoky,spicy,nutty,plain
ingredients,,,,,,,,,,,,,,,,,
진,40,0,0,2,4,9,0,0,1,0,0,0,0,0,0,0,4
럼,40,1,0,1,4,0,0,2,0,0,0,4,4,0,0,0,4
보드카,40,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,10
위스키,40,1,0,3,6,2,1,0,0,0,0,0,0,8,4,5,0
브랜디,40,2,0,1,5,1,0,2,0,0,0,0,0,6,3,6,0
데킬라,40,0,1,2,4,2,0,0,1,0,0,0,0,3,8,0,0
압생트,40,1,0,4,5,9,0,0,0,0,0,0,0,0,0,0,0
문샤인,45,0,0,2,4,0,0,0,0,0,0,0,0,0,0,0,7
오렌지 리큐르,30,8,1,2,3,1,0,8,6,0,0,0,7,0,0,0,0


In [55]:
for recipe_id, recipe in recipes.items():
    if not recipe:
        continue

    # 레시피는 원래 영어 재료명 기반이므로 바로 사용
    vector = recipe_to_vector(recipe, df_ingredient_features_eng)

    if np.sum(vector) > 0:
        X.append(vector)

        if recipe_id in df_drinks['id'].values:
            try:
                feature_values = df_drinks[df_drinks['id'] == recipe_id][drink_features].values[0]
                y.append(feature_values)
                valid_recipes.append(recipe_id)
                print(f"레시피 ID {recipe_id} 추가됨: {feature_values}")
            except KeyError as e:
                print(f"KeyError: {recipe_id}의 특성 점수를 추출하는 중 오류 발생: {e}")
        else:
            print(f"레시피 ID {recipe_id}를 drinks.csv에서 찾을 수 없음 ({recipe_id})")


경고: 다음 재료가 특성 데이터프레임에 없습니다: ice
레시피 ID 1 추가됨: [15  7  3  2  4  0  0  7  3  0  0  0  8  0  0  0  0]
경고: 다음 재료가 특성 데이터프레임에 없습니다: lemon, ice, maraschino_cherry
레시피 ID 2 추가됨: [20  8  7  2  4  0  0  5  2  0  0  0  9  0  0  8  0]
경고: 다음 재료가 특성 데이터프레임에 없습니다: ice
레시피 ID 3 추가됨: [25  6  3  2  5  0  0  6  2  0  0  0  0  0  0  0  8]
경고: 다음 재료가 특성 데이터프레임에 없습니다: orange
레시피 ID 4 추가됨: [25  5  3  4  4  0  0  4  2  0  0  0  0  0  0  0  8]
경고: 다음 재료가 특성 데이터프레임에 없습니다: s__gin, ice, es__bitters
레시피 ID 5 추가됨: [35  3  1  4  5  3  0  1  1  0  0  0  0  0  0  0  8]
경고: 다음 재료가 특성 데이터프레임에 없습니다: ice
레시피 ID 6 추가됨: [40  5  2  1  6  0  0  6  1  0  0  0  0  0  0  0  8]
경고: 다음 재료가 특성 데이터프레임에 없습니다: ice, mint
레시피 ID 7 추가됨: [20  6  4  2  5  1  0  7  3  0  0  0  0  0  0  0  0]
경고: 다음 재료가 특성 데이터프레임에 없습니다: s__gin
레시피 ID 8 추가됨: [30  7  2  1  5  0  0  6  3  0  0  0  8  0  0  0  0]
레시피 ID 9 추가됨: [40  4 10  0  4  0  0  6  7  0  0  0  0  0  0  0  0]
경고: 다음 재료가 특성 데이터프레임에 없습니다: ice
레시피 ID 10 추가됨: [30  7  2  1  5  0  0  7  3  0  0  

In [60]:
# 훈련 데이터 확인
X = np.array(X)
y = np.array(y)

In [62]:
X

array([[31.5       ,  4.3       ,  0.3       , ...,  0.        ,
         3.        ,  2.9       ],
       [25.        ,  9.        ,  1.        , ...,  0.        ,
        10.        ,  0.        ],
       [35.35714286,  2.92857143,  0.32142857, ...,  0.        ,
         0.        ,  2.75      ],
       ...,
       [37.17391304,  0.86956522,  0.7826087 , ...,  7.04347826,
         0.        ,  0.43478261],
       [36.25      ,  4.        ,  0.5       , ...,  0.        ,
         0.        ,  2.        ],
       [30.        ,  8.        ,  1.        , ...,  0.        ,
         0.        ,  0.        ]])

In [64]:
y

array([[15,  7,  3, ...,  0,  0,  0],
       [20,  8,  7, ...,  0,  8,  0],
       [25,  6,  3, ...,  0,  0,  8],
       ...,
       [14,  6,  6, ...,  0,  0,  0],
       [25,  4,  2, ...,  0,  0,  0],
       [10,  7,  5, ...,  0,  0,  0]], dtype=int64)

In [66]:
print(f"\n훈련 데이터 크기: {X.shape}, {y.shape}")


훈련 데이터 크기: (248, 17), (248, 17)


In [68]:
if len(X) == 0 or len(y) == 0:
    print("오류: 훈련 데이터가 없습니다. 데이터 불러오기와 전처리를 확인하세요.")
else:
    # 훈련 데이터 샘플 확인
    print("\n훈련 데이터 샘플:")
    for i in range(min(10, len(X))):
        print(f"X[{i}]: {X[i]}")
        print(f"y[{i}]: {y[i]}")
        print("---")

    # 모델 훈련
    print("\n모델 훈련 시작...")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    print("모델 훈련 완료")

    # 특성 중요도 확인
    print("\n특성 중요도:")
    feature_importances = model.feature_importances_
    for i, importance in enumerate(feature_importances):
        print(f"특성 {i}: {importance}")

    # 모델 저장
    joblib.dump(model, 'cocktail_quality_predictor.pkl')
    print("모델 저장됨: cocktail_quality_predictor.pkl")


훈련 데이터 샘플:
X[0]: [31.5  4.3  0.3  0.9  3.7  0.6  0.   1.8  0.   0.3  0.   2.4  5.8  0.
  0.   3.   2.9]
y[0]: [15  7  3  2  4  0  0  7  3  0  0  0  8  0  0  0  0]
---
X[1]: [25.  9.  1.  1.  4.  2.  0.  2.  0.  1.  0.  0.  8.  0.  0. 10.  0.]
y[1]: [20  8  7  2  4  0  0  5  2  0  0  0  9  0  0  8  0]
---
X[2]: [35.35714286  2.92857143  0.32142857  1.92857143  3.57142857  6.10714286
  0.          2.57142857  2.57142857  0.          0.          0.
  2.60714286  0.          0.          0.          2.75      ]
y[2]: [25  6  3  2  5  0  0  6  2  0  0  0  0  0  0  0  8]
---
X[3]: [36.81818182  0.36363636  0.          2.54545455  3.45454545  8.09090909
  0.          0.          0.81818182  0.          0.90909091  0.
  0.          0.          0.54545455  0.          3.72727273]
y[3]: [25  5  3  4  4  0  0  4  2  0  0  0  0  0  0  0  8]
---
X[4]: [15.  6.  2.  1.  4.  5.  0.  3.  1.  0.  0.  0.  7.  0.  0.  0.  0.]
y[4]: [35  3  1  4  5  3  0  1  1  0  0  0  0  0  0  0  8]
---
X[5]: [34.285714

In [70]:
def predict_cocktail(self, recipe_input):
    """레시피 입력을 받아 칵테일 특성 점수 예측"""
    # 입력이 문자열이면 파싱, 딕셔너리면 그대로 사용
    if isinstance(recipe_input, str):
        recipe = self.parse_recipe_input(recipe_input)
    else:
        recipe = recipe_input
                
    # 레시피가 비어있는지 확인
    if not recipe:
        return "오류: 유효한 재료가 입력되지 않았습니다."
            
    # 레시피 정보 출력
    print("\n입력된 레시피:")
    for ingredient, amount in recipe.items():
        print(f"  - {amount} ml {ingredient}")
            
    # 레시피를 벡터로 변환
    vector = recipe_to_vector(recipe, self.ingredient_features)
            
    # 벡터 확인
    print(f"\n변환된 벡터: {vector}")
            
    # 예측
    try:
        prediction = self.model.predict([vector])[0]
        print(f"원시 예측 값: {prediction}")
    except Exception as e:
        print(f"예측 중 오류 발생: {e}")
        return "예측 중 오류가 발생했습니다."
            
    # 결과를 딕셔너리로 변환
    result = {}
    for i, feature_name in enumerate(self.feature_names):
        # 모든 점수를 0-10 사이로 조정 (필요한 경우)
        score = min(max(prediction[i], 0), 10)
        result[feature_name] = round(score, 1)
            
    return result

In [72]:
# 칵테일 예측을 위한 클래스 정의
class CocktailPredictor:
    def __init__(self, model, ingredient_features):
        self.model = model
        self.ingredient_features = ingredient_features
        self.feature_names = [
            'abv', 'sweetness', 'sourness', 'bitterness', 'body',
            'herbal', 'coffee', 'fruity', 'citrus', 'creamy',
            'berry', 'tropical', 'sweet', 'smoky', 'spicy', 'nutty', 'plain'
        ]
        self.ingredient_mapping = ingredient_mapping
        self.special_amounts = special_amounts
        self.special_ingredients = special_ingredients

    def parse_ingredient_input(self, ingredient_str):
        amount_str, ingredient = extract_amount_and_ingredient(ingredient_str)
        std_ingredient = standardize_ingredient(ingredient)
        amount_ml = convert_to_ml(amount_str)
        if std_ingredient in self.special_ingredients and (not amount_str or amount_str == ""):
            amount_ml = self.special_amounts[std_ingredient]
        return std_ingredient, amount_ml

    def parse_recipe_input(self, recipe_str):
        recipe = {}
        if '\n' in recipe_str:
            ingredients_list = recipe_str.strip().split('\n')
        else:
            ingredients_list = recipe_str.strip().split(',')

        for ingredient_str in ingredients_list:
            ingredient_str = ingredient_str.strip()
            if ingredient_str:
                std_ingredient, amount_ml = self.parse_ingredient_input(ingredient_str)
                if std_ingredient and std_ingredient != "":
                    recipe[std_ingredient] = amount_ml

        return recipe

    def predict_cocktail(self, recipe_input):
        if isinstance(recipe_input, str):
            recipe = self.parse_recipe_input(recipe_input)
        else:
            recipe = recipe_input

        if not recipe:
            return "오류: 유효한 재료가 입력되지 않았습니다."

        print("\n입력된 레시피:")
        for ingredient, amount in recipe.items():
            print(f"  - {amount} ml {ingredient}")

        vector = recipe_to_vector(recipe, self.ingredient_features)

        print(f"\n변환된 벡터: {vector}")

        try:
            prediction = self.model.predict([vector])[0]
            print(f"원시 예측 값: {prediction}")
        except Exception as e:
            print(f"예측 중 오류 발생: {e}")
            return "예측 중 오류가 발생했습니다."

        result = {}
        for i, feature_name in enumerate(self.feature_names):
            score = min(max(prediction[i], 0), 10)
            result[feature_name] = round(score, 1)

        return result


In [74]:
ingredient_mapping

{'진': 'gin',
 '럼': 'rum',
 '보드카': 'vodka',
 '위스키': 'whiskey',
 '브랜디': 'brandy',
 '데킬라': 'tequila',
 '압생트': 'absinthe',
 '문샤인': 'moonshine',
 '오렌지 리큐르': 'orange_liqueur',
 '블루 큐라소': 'blue_curacao',
 '오렌지 주스': 'orange_juice',
 '라임 주스': 'lime_juice',
 '레몬 주스': 'lemon_juice',
 '체리 리큐르': 'cherry_liqueur',
 '크랜베리 주스': 'cranberry_juice',
 '파인애플 주스': 'pineapple_juice',
 '아몬드 리큐르': 'almond_liqueur',
 '심플 시럽': 'simple_syrup',
 '커피 리큐르': 'coffee_liqueur',
 '허브 리큐르': 'herbal_liqueur',
 '드라이 베르무트': 'dry_vermouth',
 '스위트 베르무트': 'sweet_vermouth',
 '비터스': 'bitters',
 '우유': 'milk',
 '탄산수': 'soda',
 '그레나딘 시럽': 'grenadine_syrup',
 '레몬 필': 'lemon_peel',
 '레몬 웨지': 'lemon_wedge',
 '레몬 슬라이스': 'lemon_slice',
 '라임 필': 'lime_peel',
 '라임 웨지': 'lime_wedge',
 '라임 슬라이스': 'lime_slice',
 '오렌지 필': 'orange_peel',
 '오렌지 웨지': 'orange_wedge',
 '오렌지 슬라이스': 'orange_slice',
 '체리': 'cherry',
 '애플민트': 'apple_mint',
 'mint_leaves': 'apple_mint',
 'mint_garnish': 'apple_mint',
 'orange_garnish': 'orange_peel',
 'lime_garnish': '

In [76]:
reverse_mapping = {v: k for k, v in ingredient_mapping.items()}
reverse_mapping

{'gin': '진',
 'rum': '럼',
 'vodka': '보드카',
 'whiskey': '위스키',
 'brandy': '브랜디',
 'tequila': '데킬라',
 'absinthe': '압생트',
 'moonshine': '문샤인',
 'orange_liqueur': '오렌지 리큐르',
 'blue_curacao': '블루 큐라소',
 'orange_juice': '오렌지 주스',
 'lime_juice': '라임 주스',
 'lemon_juice': '레몬 주스',
 'cherry_liqueur': '체리 리큐르',
 'cranberry_juice': '크랜베리 주스',
 'pineapple_juice': '파인애플 주스',
 'almond_liqueur': '아몬드 리큐르',
 'simple_syrup': '심플 시럽',
 'coffee_liqueur': '커피 리큐르',
 'herbal_liqueur': '허브 리큐르',
 'dry_vermouth': '드라이 베르무트',
 'sweet_vermouth': '스위트 베르무트',
 'bitters': '비터스',
 'milk': '우유',
 'soda': 'water',
 'grenadine_syrup': '그레나딘 시럽',
 'lemon_peel': '레몬 필',
 'lemon_wedge': '레몬 웨지',
 'lemon_slice': '레몬 슬라이스',
 'lime_peel': 'lime_garnish',
 'lime_wedge': '라임 웨지',
 'lime_slice': '라임 슬라이스',
 'orange_peel': 'orange_garnish',
 'orange_wedge': '오렌지 웨지',
 'orange_slice': '오렌지 슬라이스',
 'cherry': 'cherry_garnish',
 'apple_mint': 'mint_garnish',
 'ice': 'ice',
 'salt': 'salt',
 'olive_garnish': 'olive_garnish'}

In [78]:
# 한글 인덱스를 영어로 변환한 새 DataFrame 생성
df_ingredient_features_eng = df_ingredient_features_2.copy()
df_ingredient_features_eng.index = df_ingredient_features_eng.index.map(lambda kor: ingredient_mapping.get(kor, kor))

# 저장 (선택)
df_ingredient_features_eng.to_csv("../data/final/ingredient_features_eng.csv", encoding="utf-8")

# 확인
df_ingredient_features_eng

NameError: name 'df_ingredient_features_2' is not defined

In [80]:
# 샘플 데이터로 모델 테스트
print("\n=== 모델 테스트: 샘플 칵테일 예측 ===")

# 모델 인스턴스 생성
predictor = CocktailPredictor(model, df_ingredient_features_eng)

# 샘플 레시피 1: 진 마티니
gin_martini = """
1.5 oz gin
1 dash bitters
orange garnish
cherry garnish
"""

# 샘플 레시피 2: 마가리타
margarita = """
1 oz orange liqueur
1 oz brandy
lime garnish
"""

# 샘플 레시피 3: 모히토
mojito = """
1.5 oz whiskey
1 oz sweet vermouth
1 oz dry vermouth
2 dash bitters
"""

# 샘플 레시피들 테스트
sample_recipes = {
    "진 마티니": gin_martini,
    "마가리타": margarita,
    "모히토": mojito
}

for name, recipe in sample_recipes.items():
    print(f"\n{name} 예측 결과:")
    result = predictor.predict_cocktail(recipe)

    if isinstance(result, dict):
        print("\n=== 예측된 칵테일 점수 ===")
        for feature, score in result.items():
            print(f"{feature}: {score}")

        print("\n=== 시각적 표현 ===")
        for feature, score in result.items():
            bar = "■" * int(score)
            print(f"{feature.ljust(10)}: {bar} {score}")
    else:
        print(result)  # 오류 메시지 출력



=== 모델 테스트: 샘플 칵테일 예측 ===

진 마티니 예측 결과:

입력된 레시피:
  - 45.0 ml gin
  - 5.0 ml bitters
  - 5 ml orange_garnish
  - 5 ml cherry_garnish
경고: 다음 재료가 특성 데이터프레임에 없습니다: orange_garnish, cherry_garnish

변환된 벡터: [40.5  0.   0.   2.8  3.8  8.9  0.   0.   0.9  0.   0.   0.   0.   0.
  0.6  0.   3.6]
원시 예측 값: [2.71291667e+01 5.43866667e+00 2.35150000e+00 3.00466667e+00
 5.24966667e+00 8.50000000e-01 0.00000000e+00 4.90050000e+00
 3.10933333e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.81333333e+00 2.00000000e-02 0.00000000e+00 0.00000000e+00
 1.82000000e+00]

=== 예측된 칵테일 점수 ===
abv: 10
sweetness: 5.4
sourness: 2.4
bitterness: 3.0
body: 5.2
herbal: 0.8
coffee: 0.0
fruity: 4.9
citrus: 3.1
creamy: 0.0
berry: 0.0
tropical: 0.0
sweet: 3.8
smoky: 0.0
spicy: 0.0
nutty: 0.0
plain: 1.8

=== 시각적 표현 ===
abv       : ■■■■■■■■■■ 10
sweetness : ■■■■■ 5.4
sourness  : ■■ 2.4
bitterness: ■■■ 3.0
body      : ■■■■■ 5.2
herbal    :  0.8
coffee    :  0.0
fruity    : ■■■■ 4.9
citrus    : ■■■ 3.1
creamy    :  0.0
b